In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'ninapro-db1-full-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2232669%2F3734425%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240831%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240831T083126Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dbab08d231024154276a68a1a8fcd4c10571389060c4b8a5a96bb4625329cab6872989ed91ebd7741e31a8f8041093c6bb6c5a27e1829956f67b69730370f13c06eeecf15755f2e2c76c35913b824e74591ab8317c1a0c6de374726e2ad4e9d5f7845cf77900faea8755deb5d59b802d5d578923ddecdad8b08aa30f056d9590143ee332b36abb40272a012c823bc25dac85f597a86a7e21e4f9d8408a0597395718d5b2bf3a7e3dfd1b838e3a8fd41d99d027ad872ca9eb1d2e905149bb3453812685629af9669f2c26e688c120c46ab2f8fe2cf946116589a52269fdc1e1b3f5ca41f08fa41f595f2ae17ec98651ae3daebe1312a41b4bbd15723e539b09fe0'

KAGGLE_INPUT_PATH='kaggle/input'
KAGGLE_WORKING_PATH='kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount kaggle/input/ 2> /dev/null
shutil.rmtree('kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 616546734 bytes downloaded
Downloaded and uncompressed: ninapro-db1-full-dataset
Data source import complete.


In [1]:
import pandas as pd

In [ ]:
%%time
df=pd.read_csv("kaggle/input/ninapro-db1-full-dataset/Ninapro_DB1.csv")

In [2]:
import cudf

In [3]:
%%time
df=cudf.read_csv("kaggle/input/ninapro-db1-full-dataset/Ninapro_DB1.csv")

CPU times: user 2.32 s, sys: 672 ms, total: 2.99 s
Wall time: 4.05 s


In [ ]:
%%time
df=pd.read_csv("kaggle/input/ninapro-db1-full-dataset/Ninapro_DB1.csv.zip", compression='zip')